In [169]:
import pandas as pd
import numpy as np
import os 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, classification_report, precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

In [170]:
#Chargement du DF
data = pd.read_excel('Copie de Coeur.xlsx')

In [171]:
#Travailler avec une copie du data Frame pour éviter de le rechager en cas d'êrreur
df = data.copy()

In [172]:
#Afficher les informations relatives au DF
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AGE          918 non-null    int64  
 1   SEXE         918 non-null    object 
 2   TDT          918 non-null    object 
 3   PAR          918 non-null    int64  
 4   CHOLESTEROL  918 non-null    int64  
 5   GAJ          918 non-null    int64  
 6   ECG          918 non-null    object 
 7   FCMAX        918 non-null    int64  
 8   ANGINE       918 non-null    object 
 9   DEPRESSION   918 non-null    float64
 10  PENTE        918 non-null    object 
 11  CŒUR         918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [173]:
#Transformation des variables quantitatives
for col in df.drop('CŒUR', axis =1).select_dtypes(np.number).columns:
    df[col] = (df[col]-df[col].mean())/df[col].std()

In [174]:
#Vérification des résultats
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,-1.432359,homme,AA,0.410685,0.824621,-0.551041,Normal,1.382175,Non,-0.831979,Ascendant,0
1,-0.478223,femme,DNA,1.490940,-0.171867,-0.551041,Normal,0.753746,Non,0.105606,Plat,1
2,-1.750404,homme,AA,-0.129442,0.769768,-0.551041,ST,-1.524307,Non,-0.831979,Ascendant,0
3,-0.584238,femme,ASY,0.302660,0.138964,-0.551041,Normal,-1.131539,Oui,0.574398,Plat,1
4,0.051853,homme,DNA,0.950812,-0.034736,-0.551041,Normal,-0.581664,Non,-0.831979,Ascendant,0


In [175]:
#Encodage des variables qualitatives
for col in df.select_dtypes('object').columns:
    df[col] = df[col].astype('category').cat.codes

In [176]:
#Vérification des résultats
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,-1.432359,1,0,0.410685,0.824621,-0.551041,1,1.382175,0,-0.831979,0,0
1,-0.478223,0,3,1.490940,-0.171867,-0.551041,1,0.753746,0,0.105606,2,1
2,-1.750404,1,0,-0.129442,0.769768,-0.551041,2,-1.524307,0,-0.831979,0,0
3,-0.584238,0,1,0.302660,0.138964,-0.551041,1,-1.131539,1,0.574398,2,1
4,0.051853,1,3,0.950812,-0.034736,-0.551041,1,-0.581664,0,-0.831979,0,0


In [177]:
#Séparer la variable cible (coeur) et les variables explicatives
y = df['CŒUR']
x = df.drop('CŒUR', axis = 1)

In [178]:
#Création d'un objet lr de la classe LogisticRegression
lr = LogisticRegression(solver ='newton-cg', random_state =1)
clf = DecisionTreeClassifier(criterion='gini')

In [179]:
#nof_list=np.arange(1,13)            
#high_score=0
#nbre de variable a choisir pour que le resultat soit optimum
#nof=0           
#score_list =[]
#for n in range(len(nof_list)):
#    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 0)
#    model = LogisticRegression()
#    rfe = RFE(model,nof_list[n])
#    X_train_rfe = rfe.fit_transform(X_train,y_train)
#    X_test_rfe = rfe.transform(X_test)
#    model.fit(X_train_rfe,y_train)
#    score = model.score(X_test_rfe,y_test)
#    score_list.append(score)
#    if(score>high_score):
#        high_score = score
#        nof = nof_list[n]
#print("nbre optimum de variable: %d" %nof)
#print("Score avec %d ces variables: %f" % (nof, high_score))

In [180]:
cols = list(x.columns)
#choix de la classe
rfe = RFE(lr, 6)  

#Transformation des donnees en utilisant RFE
x = rfe.fit_transform(x,y)

lr.fit(X_rfe,y) 
print(x.shape)

#variables selectionnées
temp = pd.Series(rfe.support_,index = cols)
var_sel_rfe = temp[temp==True].index
print(var_sel_rfe)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_features_to_select=6 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


(918, 6)
Index(['SEXE', 'CHOLESTEROL', 'GAJ', 'ANGINE', 'DEPRESSION ', 'PENTE'], dtype='object')


In [181]:
#Subdivision du jeu de données en apprentissage et en test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state =1)
#train_set, test_set = train_test_split(df,test_size =0.2, random_state =1)

In [182]:
#Vérifier le résultat
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(734, 6) (184, 6) (734,) (184,)


In [183]:
#Apprentissage des modèles
model = lr.fit(x_train, y_train)

model_2 = clf.fit(x_train, y_train)

In [184]:
#Probabilité d'appartenance à l'une des classes (regression logistique)
predict_proba = model.predict_proba(x_test)
predict_proba[:5,:]

array([[0.32837659, 0.67162341],
       [0.18229163, 0.81770837],
       [0.03831715, 0.96168285],
       [0.10862129, 0.89137871],
       [0.35078853, 0.64921147]])

In [185]:
#Probabilité d'appartenance à l'une des classes (arbre de decision)
predict_proba_2 = model_2.predict_proba(x_test)
predict_proba_2[:5,:]

array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.33333333, 0.66666667]])

In [186]:
#Application du modèle au données de test (regression logistique)
y_pred = model.predict(x_test)
y_pred[:5]

array([1, 1, 1, 1, 1], dtype=int64)

In [187]:
#Application du modèle au données de test (arbre de decision)
y_pred_2 = model_2.predict(x_test)
y_pred_2[:5]

array([1, 1, 1, 1, 1], dtype=int64)

In [188]:
#matrice de confusion (regression logistique)
mc = confusion_matrix(y_test, y_pred)
mc

array([[ 66,   8],
       [  7, 103]], dtype=int64)

In [189]:
#matrice de confusion (arbre de decision)
mc_2 = confusion_matrix(y_test, y_pred_2)
mc_2

array([[61, 13],
       [20, 90]], dtype=int64)

In [190]:
#Taux de bonnes prédictions (regression logistique)
accuracy = accuracy_score(y_test, y_pred)
accuracy
#Conclusion : le modèle produit 92.23 % de bonne prédiction.

0.9184782608695652

In [191]:
#Taux de bonnes prédictions (arbre de decision)
accuracy_2 = accuracy_score(y_test, y_pred_2)
accuracy_2
#Conclusion : le modèle produit 79.34 % de bonne prédiction.

0.8206521739130435

In [192]:
#Sensibilité (regression logistique)
recall = recall_score(y_test, y_pred)
recall
#conclusion : lorsqu'un individu est malade, nous avons 94.54 % de chance que le modèle le prédise.

0.9363636363636364

In [193]:
#Sensibilité (arbre de decision)
recall_2 = recall_score(y_test, y_pred_2)
recall_2
#conclusion : lorsqu'un individu est malade, nous avons 72.72 % de chance que le modèle le prédise.

0.8181818181818182

In [194]:
#Précision (regression logistique)
precision = precision_score(y_test, y_pred)
precision
#Conclusion : lorsque le modèle prédit qu'un individu est malade, nous avons 92.85 % de chace que cela soit vraie.

0.9279279279279279

In [195]:
#Précision (arbre de decision)
precision_2 = precision_score(y_test, y_pred_2)
precision_2
#Conclusion : lorsque le modèle prédit qu'un individu est malade, nous avons 90.9 % de chace que cela soit vraie.

0.8737864077669902

In [196]:
#plus de détails avec classification report (regression logistique)
cr = classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.90      0.89      0.90        74
           1       0.93      0.94      0.93       110

    accuracy                           0.92       184
   macro avg       0.92      0.91      0.92       184
weighted avg       0.92      0.92      0.92       184



In [197]:
#plus de détails avec classification report (arbre de decision)
cr_2 = classification_report(y_test, y_pred_2)
print(cr_2)

              precision    recall  f1-score   support

           0       0.75      0.82      0.79        74
           1       0.87      0.82      0.85       110

    accuracy                           0.82       184
   macro avg       0.81      0.82      0.82       184
weighted avg       0.83      0.82      0.82       184



In [198]:
#Modèle Trival
df['CŒUR'].value_counts()/df.shape[0]
#Conclusion : si nous nous basons sur le hasard pour faire nos prédiction, nous aurons 55.33 % de bonne prédiction.

1    0.553377
0    0.446623
Name: CŒUR, dtype: float64

In [199]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement) (regression logistique)
model.score(x_train, y_train)

0.8474114441416893

In [200]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement) (arbre de decision)
model_2.score(x_train, y_train)

0.9850136239782016

In [201]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test) (regression logistique)
model.score(x_test, y_test)

0.9184782608695652

In [202]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test) (arbre de decision)
model_2.score(x_test, y_test)

0.8206521739130435

In [203]:
#comparaison des scores d'entrainement
if model_2.score(x_train, y_train) > model.score(x_train, y_train) :
    print("L'arbre de decision produit un meilleur score d'entrainement")
else:
    if model_2.score(x_train, y_train) < model.score(x_train, y_train) :
        print("La regression logistique produit un meilleur score d'entrainement")
    else:
        print("Les scores d'entrainement sont identiques")

L'arbre de decision produit un meilleur score d'entrainement


In [204]:
#comparaison des scores de test
if model_2.score(x_test, y_test) > model.score(x_test, y_test) :
    print("L'arbre de decision produit un meilleur score de test")
else:
    if model_2.score(x_test, y_test) < model.score(x_test, y_test) :
        print("La regression logistique produit un meilleur score de test")
    else:
        print("Les scores de test sont identiques")

La regression logistique produit un meilleur score de test
